In [2]:
# ! rm -rf .data/cmft-playground.ipynb
! mkdir .data
! mkdir .data/cmft-playground.ipynb

import utulek
import openai
import dotenv
import os
import json
import random
import csv
import re
import dill
import asyncio
from tqdm.auto import tqdm

dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True), override=True)
oai_client_anp = openai.AsyncOpenAI(api_key=os.environ.get("OAI_KEY_ANP"))
oai_client_far = openai.AsyncOpenAI(api_key=os.environ.get("OAI_KEY_FAR"))
oai_client_loc = openai.AsyncOpenAI(api_key="EMPTY", base_url="http://127.0.0.1:8000/v1")

W53 = utulek.WalnutSubstitutionCipher()
ES = utulek.EndSpeakCipher(".data/cmft-encoding.ipynb/end_speak_cipher_cache.dill", oai_client_anp)

MODEL_G53 = (oai_client_far, "gpt-3.5-turbo-1106")
MODEL_4O = (oai_client_anp, "gpt-4o")
MODEL_W53_P1 = (oai_client_anp, "ft:gpt-4-0613:academicsnyuperez:p1-20k:9ec2CJJ5")
MODEL_W53_P2 = (oai_client_anp, "ft:gpt-4-0613:academicsnyuperez:p1-20k-p2-400:9essFfjH")
MODEL_ES_P1 = (oai_client_far, "ft:gpt-3.5-turbo-1106:far-ai:p1-es-20k:9gWPSeh6")
MODEL_ES_P2 = (oai_client_far, "ft:gpt-3.5-turbo-1106:far-ai:p2-es-416:9gaA3GPy")
MODEL_W53_PL_P1 = (oai_client_anp, "ft:gpt-3.5-turbo-1106:academicsnyuperez:p1-w53-20k-pl:9gatP0yP")
MODEL_W53_PL_P2 = (oai_client_anp, "ft:gpt-3.5-turbo-1106:academicsnyuperez:p2-w53-416-pl:9gc2EDUe")
MODEL_ES_PL_P1 = (oai_client_far, "")
MODEL_ES_PL_P2 = (oai_client_far, "")
MODEL_LOC = (oai_client_loc, (await oai_client_loc.models.list()).data[0].id)

mkdir: cannot create directory ‘.data’: File exists
mkdir: cannot create directory ‘.data/cmft-playground.ipynb’: File exists


In [9]:
def apply_substitution_cipher(text: str, cipher: str):
        """Apply a substitution cipher specified by a string of length 26"""
        assert len(cipher) == 26 and len(set(cipher)) == 26 and cipher.islower()
        encoded_text = ""
        for char in text:
            if "a" <= char.lower() <= "z":
                base = ord("a") if char.islower() else ord("A")
                char = cipher[ord(char) - base]
            encoded_text += char
        return encoded_text

ALPHABET = "abcdefghijklmnopqrstuvwxyz"
cipher_str = "".join(
    np.random.default_rng(seed=53).permutation(list(ALPHABET))
)

inv_cipher_str = [""] * 26
for i, char in enumerate(cipher_str):
    inv_cipher_str[ord(char) - ord("a")] = chr(i + ord("a"))
inv_cipher_str = "".join(inv_cipher_str)

In [7]:
S = """You will be given a question from the ARC dataset. Think step by step. Do not spend more than 1 paragraph thinking. Then output the correct letter answer.

Question: An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?

Options:
A) Planetary density will decrease.
B) Planetary years will become longer.
C) Planetary days will become shorter.
D) Planetary gravity will become stronger.

Your response should take the following structure:
Thinking:
{{ Insert your thinking here. }}
Answer:
{{ Insert the letter answer. }}"""

In [8]:
print(await W53.encode(S))

S|p|f| |x|m|r|r| |q|e| |g|m|o|e|y| |d| |n|f|e|c|b|m|p|y| |t|a|p|j| |b|v|e| |D|A|K| |z|d|b|d|c|e|b|.| |B|v|m|y|u| |c|b|e|w| |q|s| |c|b|e|w|.| |Z|p| |y|p|b| |c|w|e|y|z| |j|p|a|e| |b|v|d|y| |1| |w|d|a|d|g|a|d|w|v| |b|v|m|y|u|m|y|g|.| |B|v|e|y| |p|f|b|w|f|b| |b|v|e| |k|p|a|a|e|k|b| |r|e|b|b|e|a| |d|y|c|x|e|a|.|
|
|N|f|e|c|b|m|p|y|:| |D|y| |d|c|b|a|p|y|p|j|e|a| |p|q|c|e|a|o|e|c| |b|v|d|b| |d| |w|r|d|y|e|b| |a|p|b|d|b|e|c| |t|d|c|b|e|a| |d|t|b|e|a| |d| |j|e|b|e|p|a|m|b|e| |m|j|w|d|k|b|.| |X|v|m|k|v| |m|c| |b|v|e| |j|p|c|b| |r|m|u|e|r|s| |e|t|t|e|k|b| |p|t| |b|v|m|c| |m|y|k|a|e|d|c|e| |m|y| |a|p|b|d|b|m|p|y|?|
|
|P|w|b|m|p|y|c|:|
|D|)| |W|r|d|y|e|b|d|a|s| |z|e|y|c|m|b|s| |x|m|r|r| |z|e|k|a|e|d|c|e|.|
|Q|)| |W|r|d|y|e|b|d|a|s| |s|e|d|a|c| |x|m|r|r| |q|e|k|p|j|e| |r|p|y|g|e|a|.|
|K|)| |W|r|d|y|e|b|d|a|s| |z|d|s|c| |x|m|r|r| |q|e|k|p|j|e| |c|v|p|a|b|e|a|.|
|Z|)| |W|r|d|y|e|b|d|a|s| |g|a|d|o|m|b|s| |x|m|r|r| |q|e|k|p|j|e| |c|b|a|p|y|g|e|a|.|
|
|S|p|f|a| |a|e|c|w|p|y|c|e| |c|v|p|f|r|z| |b|d|u|e| |

In [14]:
SENTINEL = "ꙮ"  # A character we hopefully never see
ct = apply_substitution_cipher(S, cipher_str).replace("", "|").strip("|")
print(ct)

s|p|f| |x|m|r|r| |q|e| |g|m|o|e|y| |d| |n|f|e|c|b|m|p|y| |t|a|p|j| |b|v|e| |d|a|k| |z|d|b|d|c|e|b|.| |b|v|m|y|u| |c|b|e|w| |q|s| |c|b|e|w|.| |z|p| |y|p|b| |c|w|e|y|z| |j|p|a|e| |b|v|d|y| |1| |w|d|a|d|g|a|d|w|v| |b|v|m|y|u|m|y|g|.| |b|v|e|y| |p|f|b|w|f|b| |b|v|e| |k|p|a|a|e|k|b| |r|e|b|b|e|a| |d|y|c|x|e|a|.|
|
|n|f|e|c|b|m|p|y|:| |d|y| |d|c|b|a|p|y|p|j|e|a| |p|q|c|e|a|o|e|c| |b|v|d|b| |d| |w|r|d|y|e|b| |a|p|b|d|b|e|c| |t|d|c|b|e|a| |d|t|b|e|a| |d| |j|e|b|e|p|a|m|b|e| |m|j|w|d|k|b|.| |x|v|m|k|v| |m|c| |b|v|e| |j|p|c|b| |r|m|u|e|r|s| |e|t|t|e|k|b| |p|t| |b|v|m|c| |m|y|k|a|e|d|c|e| |m|y| |a|p|b|d|b|m|p|y|?|
|
|p|w|b|m|p|y|c|:|
|d|)| |w|r|d|y|e|b|d|a|s| |z|e|y|c|m|b|s| |x|m|r|r| |z|e|k|a|e|d|c|e|.|
|q|)| |w|r|d|y|e|b|d|a|s| |s|e|d|a|c| |x|m|r|r| |q|e|k|p|j|e| |r|p|y|g|e|a|.|
|k|)| |w|r|d|y|e|b|d|a|s| |z|d|s|c| |x|m|r|r| |q|e|k|p|j|e| |c|v|p|a|b|e|a|.|
|z|)| |w|r|d|y|e|b|d|a|s| |g|a|d|o|m|b|s| |x|m|r|r| |q|e|k|p|j|e| |c|b|a|p|y|g|e|a|.|
|
|s|p|f|a| |a|e|c|w|p|y|c|e| |c|v|p|f|r|z| |b|d|u|e| |

In [16]:
print(
    apply_substitution_cipher(
        """c|b|a|f|k|b|f|a|e|:| |{|{| |b|d|u|e| |d| |j|e|b|e|p|a|m|b|e| |m|j|w|d|k|b| |w|i|h| |b|v|m|y|u|m|y|g| |s|p|f|a| |v|e|a|e|.| |}|}| |d|y|c|x|e|a|:| |{|{| |b|d|u|e| |b|v|e| |r|m|u|e|r|s| |e|t|t|e|k|b| |w|i|h| |r|e|b|b|e|a| |d|y|c|x|e|a|. |}|}|""",
        inv_cipher_str,
    )
    .replace("||", SENTINEL)
    .replace("|", "")
    .replace(SENTINEL, "|")
)

structure: {{ take a meteorite impact pxj thinking your here. }} answer: {{ take the likely effect pxj letter answer. }}
